# Pporownianie analizy kodu kresowego  z biblioteki z ocr

C: 2025.02.10 <br>
M: 2025.02.10


Copyright (c) 2025 Marek Kocinski

Wniosek: OCR właściwie nie działa w odniesienio do rozpoznawania kodu.

In [13]:
from pyzbar.pyzbar import decode
import cv2
import numpy
import pytesseract
from pathlib import Path


In [9]:
def ocr_barcode_reader(image):
    # Przygotowanie obrazu specjalnie pod OCR
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    
    # Użycie OCR
    text = pytesseract.image_to_string(thresh, config='--psm 6 digits')
    # Oczyszczenie wyniku
    barcode = ''.join(filter(str.isdigit, text))
    return barcode

In [10]:
class AdvancedBarcodeReader:
    def __init__(self):
        self.decoder = decode
        
    def read_barcode(self, image):
        results = {
            'pyzbar': None,
            'ocr': None,
            'final': None,
            'confidence': 0
        }
        
        # Próba odczytu przez pyzbar
        try:
            decoded = self.decoder(image)
            if decoded:
                results['pyzbar'] = decoded[0].data.decode('utf-8')
        except Exception as e:
            print(f"Pyzbar error: {e}")
            
        # Próba odczytu przez OCR
        try:
            results['ocr'] = ocr_barcode_reader(image)
        except Exception as e:
            print(f"OCR error: {e}")
            
        # Logika walidacji i porównania wyników
        if results['pyzbar'] and results['ocr']:
            if results['pyzbar'] == results['ocr']:
                results['final'] = results['pyzbar']
                results['confidence'] = 1.0
            else:
                # Sprawdzenie który wynik jest poprawny przez walidację EAN
                if self.validate_ean13(results['pyzbar']):
                    results['final'] = results['pyzbar']
                    results['confidence'] = 0.9
                elif self.validate_ean13(results['ocr']):
                    results['final'] = results['ocr']
                    results['confidence'] = 0.8
        elif results['pyzbar']:
            results['final'] = results['pyzbar']
            results['confidence'] = 0.7
        elif results['ocr']:
            results['final'] = results['ocr']
            results['confidence'] = 0.6
            
        return results

    def validate_ean13(self, code):
        # implementacja walidacji EAN-13
        pass

In [11]:
barcodereader= AdvancedBarcodeReader()

In [14]:
current_dir = Path.cwd()
imgs = current_dir.parent / 'produkty'

In [16]:
for img in imgs.glob("*.JPEG"):
    image = cv2.imread(img)
    res = barcodereader.read_barcode(image)
    print(res)


{'pyzbar': '7039010082402', 'ocr': '30174621368150085828392454840533703901008240208003300571114114908', 'final': None, 'confidence': 0}
{'pyzbar': 'http://campodelorca.com/', 'ocr': '4343402508111462640004525022004428374227', 'final': None, 'confidence': 0}
{'pyzbar': '7023026089302', 'ocr': '013725840281110704534100910028806147930409093093399651702302608930257712', 'final': None, 'confidence': 0}
{'pyzbar': None, 'ocr': '710091045060180990424955111125139149000679075913484002000484002000832172', 'final': '710091045060180990424955111125139149000679075913484002000484002000832172', 'confidence': 0.6}
{'pyzbar': None, 'ocr': '944342208779000424743', 'final': '944342208779000424743', 'confidence': 0.6}
{'pyzbar': '7039010082792', 'ocr': '43909097182206279234703901008279212841745306', 'final': None, 'confidence': 0}
{'pyzbar': 'https://polan.pl/en/recipes', 'ocr': '47040090702400194000805324084886000058256398989800983094590100837', 'final': None, 'confidence': 0}
{'pyzbar': '7038010001642', 

In [ ]:
barcodereader.